# Ejemplo 3: Comercio Internacional USA 

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

Hacemos el request de la página para luego poder parsearla con BeutifulSoup.
Si bien elegimos el 'html.parser', existen otras alternativas como 'lxml'

In [ ]:
pagina = requests.get('https://www.bea.gov/data/intl-trade-investment/international-trade-goods-and-services')

In [ ]:
soup = BeautifulSoup(pagina.content, 'html.parser')

Luego de parsear la página, vamos a buscar todas las partes de la página que contenga links y quedarán guardados en una lista.

Para poder ver cada una de esas partes, se puede iterar e imprimir el resultado.

Como suponemos que el texto del link se va a mantener, utilizamos ese string ('U.S. Trade in Goods and Services, 1960-present') para traer el link actualizado para los próximos meses y lo guardamos en una variable (link_xls).

In [ ]:
links = soup.find_all('a')

In [ ]:
for link in links:
    if 'U.S. Trade in Goods and Services, 1960-present' in link.get_text():
        link_xls = 'https://www.bea.gov' + link.get('href')

Como ya contamos con el link, hacemos la lectura del archivo directamente con Pandas.

La hoja que nos interesa es la segunda por eso le indicamos 1 en sheet_name (también le podemos pasar el nombre de la hoja y no su posición).

Como sabemos que las primeras filas no nos sirven, le pedimos que las omita.

In [ ]:
#Aca se puede hacer un request get o traerlo directamente con pandas
df_bea = pd.read_excel(link_xls, engine='openpyxl', sheet_name=1, skiprows= 6, header = None)
df_bea

Vemos que el archivo tiene muchas celdas sin ningún contenido, por lo que podemos eliminar filas/columnas enteras que no contengan valores. 

Cada vez que eliminamos filas, es conveniente que hagamos un reset del índice.

In [ ]:
#Eliminamos las columnas con todos NaNs
df_bea.dropna(how='all', axis=1, inplace=True)
#Eliminamos las filas con todos NaNs
df_bea.dropna(how='all', axis=0, inplace=True)
df_bea.reset_index(drop=True, inplace=True)
df_bea

Como queremos renombrar las columnas de acuerdo a Balance, Exports e Imports según comprenda los bienes, servicios o ambos, podemos utilizar lo que ya está en las primeras filas.

Completamos la primera fila.

In [ ]:
#Para completar los valores, se puede iterar e ir adjudicando ciertos valores
for i in range(df_bea.shape[1]):
    if df_bea.iloc[1, i] == 'Goods 1':
        df_bea.iloc[0, i-1] = df_bea.iloc[0, i]
        df_bea.iloc[0, i + 1] = df_bea.iloc[0, i]
        
#Otra alternativa
# df_bea.iloc[0].fillna(method='bfill', limit=1, inplace=True)
# df_bea.iloc[0].fillna(method='ffill', limit=1, inplace=True)
df_bea

Hacemos el cambio de los nombres de las columnas y luego determinamos en que fila comienzan los datos mensuales.

Al contar con esa información procedemos a recortar el dataframe.

In [ ]:
#Renombramos las columnas y hacemos el drop de las 3 primeras filas
df_bea.columns = df_bea.iloc[0] + ' - ' + df_bea.iloc[1]
df_bea = df_bea.iloc[3:]
#Al eliminar filas, conviene resetear el indice
df_bea.reset_index(drop=True, inplace=True)

In [ ]:
#Determino la fila en que comienzan los valores mensuales
first_row =df_bea[df_bea[df_bea.columns[0]] == 'Monthly'].index[0]

In [ ]:
df_bea = df_bea.iloc[first_row+1:]

Luego de recortar el dataframe, renombramos la primera columna que corresponde a la fecha y hacemos ciertos reemplazos para eliminar espacios innecesarios o leyendas que impiden la identificación automatica de las fechas.

In [ ]:
df_bea.rename(columns={df_bea.columns[0]:'Date'}, inplace=True)

In [ ]:
#En el mes de mayo aparece la leyenda " (R)", por lo que la reemplazamos
df_bea['Date'] = df_bea['Date'].replace(' \(R\)', '', regex=True)
#En ciertas filas hay espacios al final, por lo que tambien los limpiamos
df_bea['Date'] = df_bea['Date'].str.strip()

In [ ]:
df_bea.tail(10)

Utilizamos la función '**to_datetime**' de pandas para que cambie las fechas a un formato de Año-Mes-Día. 

Es necesario indicarle cual es el formato que figura en la columna (en nuestro caso es el año y el nombre del mes abreviado).

La indicación de '**coerce**' implica que ante la imposibilidad de identificar una fecha, nos devuelva un valor NaT.

In [ ]:
#Paso el formato a fecha
#El formato de fecha es 'YYYY (abbreviated) month name'
df_bea['Date'] = pd.to_datetime(df_bea['Date'], format= '%Y %b', errors='coerce')

In [ ]:
df_bea.dropna(how='all', axis=0, inplace=True)

Seteamos a la fecha como índice y le agregamos el país. Tener en cuenta que es necesario que estén ambas columnas (**Date** y **country**) para poder subir los datos a Alphacast.

In [ ]:
df_bea.set_index('Date', inplace=True)
df_bea['country'] = 'USA'
df_bea

#### Ahora tenemos todo listo para poder subir nuestros datos utilizando la API de Alphacast.